# first, generate a barcode read from read 2, 4, 4

basically followed this: https://github.com/alexdobin/STAR/issues/825

In [ ]:
module load mawk
for file in fastq_indrop/*_1.fastq.gz
do
    echo $file
    sample=`basename ${file%_1.fastq.gz}`
    echo $sample
    SAMPLE_R2=fastq_indrop/${sample}_2.fastq.gz
    SAMPLE_R4=fastq_indrop/${sample}_4.fastq.gz
    SAMPLE_R24=fastq_indrop/${sample}_24.fastq.gz
    echo $SAMPLE_R24
    paste <(zcat ${SAMPLE_R2}) <(zcat ${SAMPLE_R4}) | mawk '{if (NR%4==1) {print $1 " 2:4:paste"} else if (NR%4==3) {print $1} else {print $1 $2}}' | gzip > ${SAMPLE_R24} &
done

In [17]:
jobs

In [ ]:
paste \ 
    <(zcat ${SAMPLE_R2}) \
    <(zcat ${SAMPLE_R3}) \
    <(zcat ${SAMPLE_R4}) \
    | mawk '{if ((NR%4==1) && ($1 == $3) && ($3 == $5)) {print $1 " 2:3:4:paste"} else if ((NR%4==1) && ($1 != $3)) {print "singleton"} else if ((NR%4==1) && ($3 != $5)) {print "singleton"} else if (NR%4==3) {print $1} else {print $1 $2 $3}}' \
    | gzip > ${SAMPLE_R234.fastq.gz}

# then, generate bc list

In [ ]:
module load mawk
cartesian(){
awk -F '\t' \
    -v "bc1_list=$1" \
    -v "bc2_list=$2" '
    {
        if ( FILENAME == bc1_list ) {
            BC1[$0];
        }
        else if ( FILENAME == bc2_list ) {
             BC2[$0];
        }
    } END {
        for (bc1 in BC1) {
           for (bc2 in BC2) {
               print bc1 bc2
           }
        }
     } ' $1 $2 > $3
}


In [ ]:
cat 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list.txt | tr 'ACGT' 'TGCA' | rev > 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list_revcomp.txt

In [ ]:
cartesian 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list.txt 20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list_revcomp.txt 20220127_indrop_barcodes_kalish_384x384/full_whitelist

# then map

pos 1-8: bc1  
pos 9-16: bc2 (reverse complement of whitelist)  
pos 17-22: UMI

In [22]:
star_solo_indrop_rna_mapping_unique_cellfilter () {
    local fastq_R1_filename="${1}";
    local fastq_R2_filename="${2}";
    local bam_filename="${3}";
    # 3 barcode lists:
    local whitelist_part1_filename='/lustre1/project/stg_00002/lcb/fderop/data/00000000_barcodes/20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list.txt';
    local whitelist_part2_filename='/lustre1/project/stg_00002/lcb/fderop/data/00000000_barcodes/20220127_indrop_barcodes_kalish_384x384/gel_barcode2_list_revcomp.txt';
    star_reference_dir='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna';
    sjdbgtf='/lustre1/project/stg_00002/lcb/fderop/data/00000000_genomes/mm10_STAR_2.7.8_premrna/genes.gtf'
    #module load STAR/2.7.5b-foss-2018a;
    #module load SAMtools/1.10-foss-2018a;
    # Map with STAR solo
    /lustre1/project/stg_00002/lcb/fderop/scripts/STAR-2.7.8a/bin/Linux_x86_64/STAR \
        --runThreadN 36 \
        --runMode alignReads \
        --outSAMtype BAM SortedByCoordinate \
        --sysShell /bin/bash \
        --genomeDir "${star_reference_dir}" \
        --readFilesIn "${fastq_R1_filename}" "${fastq_R2_filename}" \
        --readFilesCommand 'gzip -c -d' \
        --soloCBwhitelist "${whitelist_part1_filename}" "${whitelist_part2_filename}" \
        --soloType CB_UMI_Complex \
        --soloCBposition 0_0_0_7 0_8_0_15 \
        --soloUMIposition 0_16_0_21 \
        --sjdbGTFfile $sjdbgtf \
        --soloCellFilter CellRanger2.2 2000 0.99 10 \
        --soloCBmatchWLtype 1MM \
        --outFilterMultimapNmax 1 \
        --outSAMattributes NH HI AS nM CB UB CR CY UR UY \
        --outFileNamePrefix ${bam_filename%bam} \
        --outReadsUnmapped Fastx \
        --quantMode GeneCounts \
        --bamRemoveDuplicatesType UniqueIdentical \
        --soloFeatures Gene GeneFull
        # Index BAM file.
    # samtools index "${bam_filename%bam}Aligned.sortedByCoord.out.bam"
}

Now we run this star solo script in a for loop on all the R2 fastq files in `fastq/`

In [23]:
dir=demultiplexed_indrop
mkdir $dir
for sample in fastq_indrop/*_1.fastq.gz
do
    NAME=${sample%_1.fastq.gz}
    OUT=${NAME##*/}
    echo $NAME
    star_solo_indrop_rna_mapping_unique_cellfilter \
        ${NAME}_1.fastq.gz \
        ${NAME}_24.fastq.gz \
        $dir/$OUT.bam
done

mkdir: cannot create directory ‘demultiplexed_indrop’: File exists
fastq_indrop/SRR10545068
Jan 28 16:03:41 ..... started STAR run
Jan 28 16:03:41 ..... loading genome
Jan 28 16:04:09 ..... processing annotations GTF
Jan 28 16:04:15 ..... started mapping
Jan 28 16:14:44 ..... finished mapping
Jan 28 16:14:47 ..... started Solo counting
Jan 28 16:19:06 ..... finished Solo counting
Jan 28 16:19:06 ..... started sorting BAM
Jan 28 16:20:43 ..... finished successfully
fastq_indrop/SRR10545069
Jan 28 16:20:53 ..... started STAR run
Jan 28 16:20:53 ..... loading genome
Jan 28 16:21:11 ..... processing annotations GTF
Jan 28 16:21:12 ..... started mapping
Jan 28 16:29:11 ..... finished mapping
Jan 28 16:29:15 ..... started Solo counting
Jan 28 16:30:12 ..... finished Solo counting
Jan 28 16:30:12 ..... started sorting BAM
Jan 28 16:31:22 ..... finished successfully
fastq_indrop/SRR10545070
Jan 28 16:31:29 ..... started STAR run
Jan 28 16:31:29 ..... loading genome
Jan 28 16:31:44 ..... proces

Now, we can get a summary of the statistics below:

In [24]:
for sample in $dir/*/Gene/Summary.csv
do
    echo ${sample#*/}
    cat $sample
    cut -d, -f2 $sample > ${sample%/Gene/Summary.csv}/Gene/Summary_numbers.csv
    printf "\n"
done

SRR10545068.Solo.out/Gene/Summary.csv
Number of Reads,136252173
Reads With Valid Barcodes,0.440879
Sequencing Saturation,0.266368
Q30 Bases in CB+UMI,0.832998
Q30 Bases in RNA read,0.800453
Reads Mapped to Genome: Unique+Multiple,0.488978
Reads Mapped to Genome: Unique,0.488978
Reads Mapped to Transcriptome: Unique+Multipe Genes,0.301477
Reads Mapped to Transcriptome: Unique Genes,0.293621
Estimated Number of Cells,14363
Reads in Cells Mapped to Unique Genes,24349722
Fraction of Reads in Cells,0.608644
Mean Reads per Cell,1695
Median Reads per Cell,1432
UMIs in Cells,17563343
Mean UMI per Cell,1222
Median UMI per Cell,1044
Mean Genes per Cell,896
Median Genes per Cell,806
Total Genes Detected,21125

SRR10545069.Solo.out/Gene/Summary.csv
Number of Reads,103036640
Reads With Valid Barcodes,0.381989
Sequencing Saturation,0.196112
Q30 Bases in CB+UMI,0.732578
Q30 Bases in RNA read,0.711272
Reads Mapped to Genome: Unique+Multiple,0.450167
Reads Mapped to Genome: Unique,0.450167
Reads Mapped